In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper



ERROR: Could not find a version that satisfies the requirement wikipedia (from versions: none)
ERROR: No matching distribution found for wikipedia


In [18]:
## now over here we will be making out a wrapper that can make sure that we can first set up the wrapper around the wikipida with the content taht we want
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
##making the tool setting it up with this wrapper to make sure we use it to interact with the wikipedia
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [16]:
##this one is the custom tool for this requirement for the webbase loeadr
##the above ones are used for the already existing apis
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embeddings)
retriever=db.as_retriever()
retriever





VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000254FFD21A90>, search_kwargs={})

In [12]:
##making a tools that can retrive the info on the langsmith
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search for info about langsmith. for any question about langsmith u must use this tool")

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

Arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv= ArxivQueryRun(api_wrapper=Arxiv_wrapper)
arxiv.name

'arxiv'

In [19]:
##now combining / chaining the above made tools
tools=[wiki,arxiv,retriever_tool]

In [20]:
##now to use the agents in order to perform the search on this combined tools thing lets make a new llm
from dotenv import load_dotenv

load_dotenv()
import os

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [21]:
##u can make your own prompt using promttemplate
##but over here we will make a new prompt and get that from hub
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
#now lets create the agent in order to perform the retrieval
from langchain.agents import create_openai_functions_agent
agent=create_openai_functions_agent(llm,tools,prompt)

In [27]:
##to execute the agent operation we use agent executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [28]:
agent_executer.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Get started with LangSmith - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEv

{'input': 'Tell me about Langsmith',
 'output': "LangSmith is a platform for building production-grade LLM applications.  It allows you to monitor and evaluate your application, enabling faster and more confident deployments.  It's framework-agnostic, meaning you can use it with or without LangChain's open-source frameworks.\n"}